In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
import numpy as np
import random
import pandas as pd
import io
import  urllib
import ssl

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

## The following files have been created separately and have been uploaded to project workspace
    DubPostCodes.csv #contains centroid location co-ordinates for Dublin PostCodes
    DubNhood.csv #contains centroid location co-ordinates for Dublin Neibourhooods
    DubPPR2018.csv #Filtered and cleansed records for properties sold in 2018 in Dublin county only
    dublinplus_geo.json #custom geojson file for Dublin including larger Co Dublin neighbourhoods

In [4]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 441.10 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 674.37 kB/s


In [5]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

r = urllib.request.urlopen('https://propertypriceregister.ie/website/npsra/ppr/npsra-ppr.nsf/Downloads/PPR-ALL.zip/$FILE/PPR-ALL.zip', context=ctx)
df_ppr_all = pd.read_csv(io.BytesIO(r.read()), compression='zip', sep=',', header=0, encoding='ISO-8859-1')
df_ppr_all.columns=['SaleDate','Address','PostCode','County','Price','FullMarketPrice','VATExcl','PropDesc','SizeDesc']
df_ppr_all.head()

,SaleDate,Address,PostCode,County,Price,FullMarketPrice,VATExcl,PropDesc,SizeDesc
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,"343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,"185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,"438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN
3,04/01/2010,"1 The Haven, Mornington",NaN,Meath,"400,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
4,04/01/2010,"11 Melville Heights, Kilkenny",NaN,Kilkenny,"160,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN


In [6]:
df_ppr = df_ppr_all[df_ppr_all.County == 'Dublin']
df_ppr.reset_index(inplace=True, drop=True)

In [7]:
def sizedesc(row):
    if row['SizeDesc'] == 'less than 38 sq metres':
        val = '001 - 038'
    elif row['SizeDesc'] == 'greater than or equal to 38 sq metres and less than 125 sq metres' or row['SizeDesc'] == 'n?os l? n? 38 m?adar cearnach':
        val = '038 - 124'
    elif row['SizeDesc'] == 'greater than or equal to 125 sq metres' or row['SizeDesc'] == 'greater than or equal to 125 sq metres':
        val = '125 - 999'
    else:
        val = '000 - 000'
    return val

df_ppr['SizeDesc'] = df_ppr.apply(sizedesc, axis=1)

df_ppr['SizeDesc'].value_counts()

000 - 000    92291
038 - 124    13908
125 - 999     1729
001 - 038      938
Name: SizeDesc, dtype: int64

In [9]:
def propdesc(row):
    if row['PropDesc'] == 'New Dwelling house /Apartment':
        val = 'New'
    else:
        val = 'Old'
    return val

df_ppr['PropDesc'] = df_ppr.apply(propdesc, axis=1)
df_ppr['PropDesc'].value_counts()

Old    90782
New    18084
Name: PropDesc, dtype: int64

In [10]:
df_ppr['SaleDate'] = pd.to_datetime(df_ppr['SaleDate'],format="%d/%m/%Y")
df_ppr['PostCode']=df_ppr['PostCode'].fillna('') 
df_ppr['PostCode'] = df_ppr.apply(
    lambda row: row['Address'].rsplit(',',1)[1].strip(' ').title() if (row['PostCode'] == '') else row['PostCode'].capitalize(),
    axis=1,ignore_failures=True
)
df_ppr['Address']=df_ppr['Address'].str.title()
df_ppr.head(5)

,SaleDate,Address,PostCode,County,Price,FullMarketPrice,VATExcl,PropDesc,SizeDesc
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Co.Dublin,Dublin,"343,000.00",No,No,Old,000 - 000
1,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin 14,Dublin,"438,500.00",No,No,Old,000 - 000
2,2010-01-04,"12 Sallymount Avenue, Ranelagh",Ranelagh,Dublin,"425,000.00",No,No,Old,000 - 000
3,2010-01-04,"206 Philipsburgh Avenue, Marino, Dublin 3",Dublin 3,Dublin,"430,000.00",No,No,Old,000 - 000
4,2010-01-04,"22 Laverna Way, Castleknock, Dublin 15",Dublin 15,Dublin,"355,000.00",No,No,Old,000 - 000


In [13]:
df_ppr['Price']=df_ppr.Price.str.encode('utf-8').str.decode('ascii', 'ignore')
df_ppr['Price']=df_ppr['Price'].str.replace(',','').replace(' ','')
df_ppr['Price']=df_ppr['Price'].astype(float)
df_ppr['FullMarketPrice'] = df_ppr['FullMarketPrice'].map({'Yes': 1, 'No': 0})
df_ppr['VATExcl'] = df_ppr['VATExcl'].map({'Yes': 1, 'No': 0})
df_ppr.head(5)

,SaleDate,Address,PostCode,County,Price,FullMarketPrice,VATExcl,PropDesc,SizeDesc
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Co.Dublin,Dublin,343000.00,0,0,Old,000 - 000
1,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin 14,Dublin,438500.00,0,0,Old,000 - 000
2,2010-01-04,"12 Sallymount Avenue, Ranelagh",Ranelagh,Dublin,425000.00,0,0,Old,000 - 000
3,2010-01-04,"206 Philipsburgh Avenue, Marino, Dublin 3",Dublin 3,Dublin,430000.00,0,0,Old,000 - 000
4,2010-01-04,"22 Laverna Way, Castleknock, Dublin 15",Dublin 15,Dublin,355000.00,0,0,Old,000 - 000


In [14]:
df_ppr.PostCode = df_ppr.PostCode.replace('\s+', ' ', regex=True)

pattern='Abbey Street|Amiens street|Capel street|Dublin1|Dorset street|Henry street|Mountjoy square|Marlborough street|North wall|O\'connell street|Parnell square|Talbot street|Dublin 1 dublin 1'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 1')

pattern='Merrion square|Temple bar|Grafton street|Dame street|Leeson street|Wexford street|St.stephen\'s green|St.stephens green|Dublin 2 dublin 2'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 2')

pattern='Ballybough|North strand|Clonliffe|Clontarf|Dollymount|East wall|East point|Fairview|Killester|Marino|Dublin 3al|Dublin 3 dublin 3'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 3')

pattern='Ballsbridge|Belfield|Donnybrook|Irishtown|Merrion|Pembroke|Ringsend|Sandymount|Landsdowne road|South lotts road|Dublin 4 dublin 4'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 4')

pattern='Artane|Coolock|Harmonstown|Kilbarrack|Raheny|Dublin 5 dublin 5'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 5')

pattern='Milltown|Ranelagh|Rathmines|Dartry|Rathgar|Harrolds cross dublin 6|Dublin 6 dublin 6'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 6')

pattern='Harold\'s cross|Harolds cross|Templeogue|Kimmage|Terenure|Dublin 6w|Dublin 6W dublin 6w'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 6W')

pattern='Arbour hill|Ashtown|Broadstone|Cabra|Grangegorman|Oxmantown|Phibsboro|Smithfield|Stoneybatter|Ratoath road ashtown|Dublin 7 dublin 7'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 7')

pattern='Dolphin\'s barn|Inchicore|Islandbridge|Kilmainham|Portobello|Rialto|South circular road|Phoenix park|Liberties|Dublin 8 dublin 8'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 8')

pattern='Beaumont|Donnycarney|Drumcondra|Santry|Whitehall|Dublin 9 road|Dublin 9 dublin 9'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 9')

pattern='Ballyfermot|Cherry orchard|Dublin 10 dublin 10'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 10')

pattern='Ballymun|Glasnevin|Finglas|Balygall|Cappagh|Poppintree|Sillogue|Dublin 11 dublin 11'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 11')

pattern='Bluebell|Crumlin|Drimnagh|Greenhills|Perrystown|Terenure|Walkinstown|Dublin 12 dublin 12'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 12')

pattern='Baldoyle|Bayside|Donaghmede|Clongriffin|Sutton|Howth|Ayrfield|Dublin 13 dublin 13'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 13')

pattern='Churchtown|Clonskeagh|Dundrum|Goatstown|Rathfarnham|Windy Arbour|Dublin 14 dublin 14'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 14')

pattern='Blanchardstown|Carpenterstown|Castaheaney|Castleknock|Coolmine|Clonsilla|Corduff|Hansfield|Hartstown|Huntstown|Mulhuddart|Ongar|Tyrrelstown|Tyrellstown|Dublin 15 dublin 15'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 15')

pattern='Ballinteer|Ballyboden|Kilmashogue|Knocklyon|Rockbrook|Dublin 16 dublin 16'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 16')

pattern='Balgriffin|Belcamp|Darndale|Priorswood|Dublin 17 dublin 17'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 17')

pattern='Cabinteely|Carrickmines|Clonshaugh|Deansgrange|Foxrock|Kilternan|Sandyford|Shankill|Shankhill|Stepaside|Leopardstown|Dublin 18 dublin 18'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 18')

pattern='Chapelizod|Palmerstown|Dublin 20 dublin 20'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 20')

pattern='Clondalkin|Rowlagh|Quarryvale|Liffey Valley|Neilstown|Ronanstown|Dublin 22 dublin 22'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 22')

pattern='Firhouse|Jobstown|Kilnamanagh|Tallaght|Dublin 24 dublin 24'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Dublin 24')

pattern='Adamstown|Balbriggan|Barnageeragh|Blackrock|Cloghran|County dublin|Dun laoghaire|Dunlaoghaire|Forrest road|Feltrim rd|Glasthule|Kingswood|Lucan|Malahide|Skerries|Swords'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Co Dublin')

pattern='Ballbriggan|Ballycullen|Citywest|Clare hall|Co.Dublin.|Dalkey|Glenamuck road|Killiney|Kiltipper|Lusk|Newcastle|Portmarnock|Portrane|Rush|Saggart|Sandycove|Skerries|Stillorgan'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Co Dublin')

pattern='Balrothery|Booterstown|Co dublin|Co.dublin|Co.Dublinnue|Donabate|Dun Laoghaire|Dun-laoghaire|Glenageary|Kinsealy|Monkstown|Naul|Rathcoole|The naul|Upper glenageary road'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Co Dublin')

pattern='Ballybrack|Co.Dublin|Co. dublin|Co.dublin  Co.dublin|Co.Dublin ave|Co.Dublin road|Co.Dublin rd|Dun laoighre|Forrest road swords|Hollystown|Loughlinstown|Meakstown|Mount merrion|Sallynoggin'
df_ppr['PostCode']=df_ppr['PostCode'].str.replace(pattern,'Co Dublin')

df_ppr['PostCode'].value_counts().sort_index()

& 174 -180 Harolds Cross Road              1
.                                          8
1 Dublin 5 Rd Dublin 5                     1
1 Jervis Street                            3
1/2 Henrietta Street                       1
10 Seafort Avenue                          1
10B/11 East Essex Street                   1
113 Haroldscross Rd Dublin 6W              1
117 - 126 Upper Sheriff Street             5
117 To 126 Upper Sheriff Street            1
119 - 125 Thomas Street                    1
126/133 James Street                       2
13 15 Summerhill Parade Dublin 1           1
13-17 Dublin 4 Road                        1
13/17 Dublin 4 Road                        1
132 James Street                           1
149 North Strand Rd                        6
149 North Strand Road                      1
16 Kings Inn Street                        1
16 Kings Inns Street                       1
167 Co Dublin Avenue                       1
17 Cook St                                 1
172 North 

In [15]:
df_incl = df_ppr.PostCode.value_counts().nlargest(24)
df_filt = df_ppr[df_ppr.PostCode.isin(df_incl.index)]
df_filt.head(20)
#df_ppr[df_ppr.PostCode in df_incl.index]

,SaleDate,Address,PostCode,County,Price,FullMarketPrice,VATExcl,PropDesc,SizeDesc
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Co Dublin,Dublin,343000.00,0,0,Old,000 - 000
1,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin 14,Dublin,438500.00,0,0,Old,000 - 000
2,2010-01-04,"12 Sallymount Avenue, Ranelagh",Dublin 6,Dublin,425000.00,0,0,Old,000 - 000
3,2010-01-04,"206 Philipsburgh Avenue, Marino, Dublin 3",Dublin 3,Dublin,430000.00,0,0,Old,000 - 000
4,2010-01-04,"22 Laverna Way, Castleknock, Dublin 15",Dublin 15,Dublin,355000.00,0,0,Old,000 - 000
5,2010-01-04,"23 Elton Park, Sandycove",Co Dublin,Dublin,1800000.00,0,0,Old,000 - 000
6,2010-01-04,"29 Carysfort Woods, Blackrock",Co Dublin,Dublin,547500.00,0,0,Old,000 - 000
7,2010-01-04,"33 Ragian Road, Ballsbridge, Dublin 4",Dublin 4,Dublin,700000.00,0,0,Old,000 - 000
9,2010-01-04,"49 Ballynakelly Green, Newcastle",Co Dublin,Dublin,499600.00,0,1,New,000 - 000
10,2010-01-04,"5 Clancy Road, Finglas, Dublin 11",Dublin 11,Dublin,205000.00,0,0,Old,000 - 000


In [16]:
df_filt['PostCode'].value_counts()

Co Dublin    16905
Dublin 15     9186
Dublin        6692
Dublin 18     5958
Dublin 24     4766
Dublin 8      4518
Dublin 4      4428
Dublin 7      4311
Dublin 14     4177
Dublin 9      4085
Dublin 11     3942
Dublin 13     3928
Dublin 6      3722
Dublin 12     3665
Dublin 3      3380
Dublin 16     3336
Dublin 5      2905
Dublin 1      2640
Dublin 22     2162
Dublin 6W     1784
Dublin 2      1541
Dublin 20     1006
Dublin 10      988
Dublin 17      885
Name: PostCode, dtype: int64

In [17]:
import random
def popcoord(Address):
    geolocator = Nominatim(user_agent='cstone'+str(random.randint(1,101)))
    loc=geolocator.geocode(Address)
    if loc is None:
        return '.'
    else:
        return str(loc.latitude)+', ' + str(loc.longitude)

In [18]:
df_filt.reset_index(drop=True, inplace=True)

In [24]:
df_filt2018 = df_filt[df_filt.SaleDate > '2017-12-31'].reset_index(drop=True)
#df_filt['Price']=df_filt['Price'].astype(float)
df_filt2018.head()

,SaleDate,Address,PostCode,County,Price,FullMarketPrice,VATExcl,PropDesc,SizeDesc
0,2018-01-02,"1 Abbey St, Howth, Dublin",Dublin 7,Dublin,710000.00,0,0,Old,000 - 000
1,2018-01-02,"1 Roseville, Lower Road, Shankill",Dublin 18,Dublin,627230.70,0,0,Old,000 - 000
2,2018-01-02,"19 Ulverton Rd, Dalkey, Dublin",Dublin,Dublin,1130000.00,0,0,Old,000 - 000
3,2018-01-02,"236 Phibsboro Rd, Dublin 7, Dublin",Dublin 7,Dublin,277000.00,0,0,Old,000 - 000
4,2018-01-02,"3 College Court, Portrane Road, Donabate",Co Dublin,Dublin,348017.62,0,0,Old,000 - 000


In [ ]:
#df_filt['Location']=""
import time
for i in range(11360, -1, -1):
    if df_filt2018.iloc[i]['Location']=='.':
        addr=df_filt2018.iloc[i]['Address']+' '+df_filt2018.iloc[i]['Neighbourhood']+', '+df_filt2018.iloc[i]['PostCode']+', Ireland'
        print(addr)
        locn=popcoords(addr)
        df_filt2018.at[i,'Location']=locn
        time.sleep(.500)

In [ ]:
#df_filt[df_filt.Location=='.'].tail(5)
df_filt2018[df_filt2018.Location=='.'].head()

In [ ]:
import requests
import json
from collections import namedtuple

def popcoords(Address):
    url = "https://eu1.locationiq.com/v1/search.php"

    data = {
        'key': LocIQKey,
        'q': Address,
        'format': 'json'
    }
    response = requests.get(url, params=data)
    x = json.loads(response.text, object_hook=lambda d: namedtuple('X', d.keys(),rename=True)(*d.values()))
    if x is not None:
        if x[0].lat is not None:
            return str(x[0].lat)+', ' + str(x[0].lon)
        else:
            return '.'
    else:
        return '.'

In [ ]:
#df_filt.at[13,'Location']="53.29067, -6.25714"
popcoords('4 Palmers Lawn, Palmerstown Manor, Palmerstown Dublin 20')

In [ ]:
df_filt2018.at[323,'Location']='53.476676, -6.318977'

In [ ]:
df_filt2018['SaleDate'][df_filt2018.Location=='.'].count()

In [ ]:
csv_file = project.get_file("DubPPRFilt.csv")
csv_file.seek(0)
df_filt = pd.read_csv(csv_file)
df_filt['SaleDate'] = pd.to_datetime(df_filt['SaleDate']) #,format="%d/%m/%Y")
df_filt['Location']=df_filt['Location'].fillna('')
df_filt.tail(5)

In [ ]:
df_filt.reset_index(drop=True, inplace=True)

In [ ]:
df_filt2018.dtypes

In [ ]:
#f_filt[df_filt['Location']=='53.078731, -7.798618'].Location = ''
df_filt['Location'] = df_filt['Location'].str.strip().replace("53.078731, -7.798618","") # Center of Ireland, incorrect for Dublin!

In [ ]:
df_filt=df_filt[['SaleDate', 'Address', 'PostCode', 'County', 'Location','Price', 'FullMarketPrice','VATExcl', 'PropDesc', 'SizeDesc']]

In [ ]:
df_postcode.columns=['PostCode','Dummy','Location']

In [ ]:
df_postcode['Location'][df_postcode.PostCode=='Dublin 7']='53.361571, -6.291803'

In [ ]:
df_postcode

In [ ]:
csv_fil = project.get_file("DubNhood.csv")
csv_fil.seek(0)
df_nhood = pd.read_csv(csv_fil)
df_nhood.head()

In [ ]:
import time
for i in range(0, -1, -1):
    if df_nhood.iloc[i]['Location']=='':
        addr=df_nhood.iloc[i]['Neighbourhood']+', '+df_nhood.iloc[i]['PostCode']+', Ireland'
        print(addr)
        locn=popcoord(addr)
        df_nhood.at[i,'Location']=locn
        time.sleep(.500)

In [ ]:
df_nhood[df_nhood.Location=='']

In [ ]:
#df_filt2018[df_filt2018.Neighbourhood.str.lower().isin(df_filt2018.Address.str.lower())].head()
#df_filt2018.head(10)
#'Dublin'.isin('236 Phibsboro Rd, , Dublin')
#df_filt2018['Address'] = df_filt2018.apply(lambda row: row['Address'].replace(row['Neighbourhood'],''),axis=1,ignore_failures=True)
df_filt2018['Address'] = df_filt2018.apply(lambda row: row['Address'].rstrip(', ,'),axis=1)
#df_filt2018["Neighbourhood"] = df_filt2018["Neighbourhood"].str.replace(',','')
#df_filt2018['Address'] = df_filt2018.apply(lambda row: row['Address'].replace('Dublin',''),axis=1,ignore_failures=True)

In [36]:
#df_filt2018.at[11359,'Neighbourhood']='Cabra'
#df_filt2018.Neighbourhood.value_counts()